In [1]:
# links:  https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others
#https://stackoverflow.com/questions/24614474/pandas-merge-on-name-and-closest-date

In [2]:
import pandas as pd
import sqlite3
from datetime import datetime

#We'll use firelynx's tables:
presidents = pd.DataFrame({"name": ["Bush", "Obama", "Trump"],
                           "president_id":[43, 44, 45]})
terms = pd.DataFrame({'start_date': pd.date_range('2001-01-20', periods=5, freq='48M'),
                      'end_date': pd.date_range('2005-01-21', periods=5, freq='48M'),
                      'president_id': [43, 43, 44, 44, 45]})
war_declarations = pd.DataFrame({"date": [datetime(2001, 9, 14), datetime(2003, 3, 3)],
                                 "name": ["War in Afghanistan", "Iraq War"]})
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
terms.to_sql('terms', conn, index=False)
presidents.to_sql('presidents', conn, index=False)
war_declarations.to_sql('wars', conn, index=False)

qry = '''
    select  
        start_date PresTermStart,
        end_date PresTermEnd,
        wars.date WarStart,
        presidents.name Pres
    from
        terms join wars on
        date between start_date and end_date join presidents on
        terms.president_id = presidents.president_id
    '''
df = pd.read_sql_query(qry, conn)

In [3]:
presidents

,name,president_id
0,Bush,43
1,Obama,44
2,Trump,45


In [4]:
terms

,start_date,end_date,president_id
0,2001-01-31,2005-01-31,43
1,2005-01-31,2009-01-31,43
2,2009-01-31,2013-01-31,44
3,2013-01-31,2017-01-31,44
4,2017-01-31,2021-01-31,45


In [5]:
war_declarations

,date,name
0,2001-09-14,War in Afghanistan
1,2003-03-03,Iraq War


In [6]:
df

,PresTermStart,PresTermEnd,WarStart,Pres
0,2001-01-31 00:00:00,2005-01-31 00:00:00,2001-09-14 00:00:00,Bush
1,2001-01-31 00:00:00,2005-01-31 00:00:00,2003-03-03 00:00:00,Bush


In [7]:
cat = ['790036', '860081']

data_dir = '../Data/big_cats/Access_DB_table_exports/'
biochem = pd.read_excel(data_dir + 'Biochemistry ReportsTZ_ZV_WA_SA_MG_DM.xlsx')
biochem['Date'] = pd.to_datetime(biochem['Date'])
biochem = biochem[biochem['ARKS Number'].isin(cat)][['ARKS Number', 'Date', 'Creatinine (umol/L)']]
biochem['ARKS No'] = biochem['ARKS Number']
biochem.drop('ARKS Number', axis = 1, inplace=True)

data_dir = '../Data/big_cats/Access_DB_table_exports/'
haem = pd.read_excel(data_dir + 'Haematology ReportsTZ_ZV_MG.xlsx')
haem['Date'] = pd.to_datetime(haem['Date'])
haem = haem[haem['ARKS No'].isin(cat)][['ARKS No', 'Date', 'RBC (x 10^12/L)']]

data_dir = '../Data/big_cats/Access_DB_table_exports/'
urine = pd.read_excel(data_dir + 'UrinalysisTZ_ZV_WA_SA_MG_DM.xlsx')
urine['Date of Sample Analysis'] = pd.to_datetime(urine['Date of Sample Analysis'])
urine = urine[urine['ARKS No'].isin(cat)][['ARKS No', 'Date of Sample Analysis', 'Urine Specific Gravity']]
urine['Date'] = urine['Date of Sample Analysis']
urine.drop('Date of Sample Analysis', axis = 1, inplace=True)

In [8]:
# do some im,portant stuff
haem = haem.sort_values(by=['Date'])
biochem = biochem.sort_values(by=['Date'])
urine = urine.sort_values(by=['Date'])

biochem['biochem date'] = biochem['Date']
haem['haem date'] = haem['Date']
urine['urine date'] = urine['Date']

In [9]:
biochem

,Date,Creatinine (umol/L),ARKS No,biochem date
2,1987-06-10,170.0,790036,1987-06-10
7,1992-04-03,208.0,860081,1992-04-03
3,1993-01-28,135.0,790036,1993-01-28
4,1993-05-10,143.0,790036,1993-05-10
8,1993-10-06,193.0,860081,1993-10-06
5,1996-07-18,110.0,790036,1996-07-18
6,1996-07-18,91.8,790036,1996-07-18
9,2000-08-24,187.0,860081,2000-08-24
10,2003-03-19,263.0,860081,2003-03-19
11,2007-03-29,260.0,860081,2007-03-29


In [10]:
haem

,ARKS No,Date,RBC (x 10^12/L),haem date
97,790036,1987-06-10,NaN,1987-06-10
178,790036,1988-06-08,8.83,1988-06-08
266,790036,1989-04-07,NaN,1989-04-07
563,790036,1993-01-28,NaN,1993-01-28
562,790036,1993-05-10,NaN,1993-05-10
491,790036,1996-07-18,NaN,1996-07-18


In [11]:
urine

,ARKS No,Urine Specific Gravity,Date,urine date
11,790036,1.080,1989-08-01,1989-08-01
10,790036,NaN,1990-09-14,1990-09-14
9,790036,>1.050,1991-03-21,1991-03-21
8,790036,>1.050,1992-07-06,1992-07-06
7,790036,1.038,1992-07-13,1992-07-13
21,860081,1.072,1992-07-31,1992-07-31
6,790036,2.068,1993-01-28,1993-01-28
20,860081,1.068,1993-09-20,1993-09-20
15,860081,1.060,1993-09-24,1993-09-24
12,860081,1.074,1993-09-30,1993-09-30


In [12]:
# the magic happens below, check the direction and tolerance arguments
# if you want you can make a maximum tolerance on which to merge data

tol = pd.Timedelta('3 day')
#df3 = pd.merge_asof(left=df1,right=df2,right_index=True,left_index=True,direction='nearest',tolerance=tol)

df3 = pd.merge_asof(by=['ARKS No'], left=biochem,right=haem, left_on='Date', right_on='Date', direction='nearest',tolerance=tol)
df3

,Date,Creatinine (umol/L),ARKS No,biochem date,RBC (x 10^12/L),haem date
0,1987-06-10,170.0,790036,1987-06-10,NaN,1987-06-10
1,1992-04-03,208.0,860081,1992-04-03,NaN,NaT
2,1993-01-28,135.0,790036,1993-01-28,NaN,1993-01-28
3,1993-05-10,143.0,790036,1993-05-10,NaN,1993-05-10
4,1993-10-06,193.0,860081,1993-10-06,NaN,NaT
5,1996-07-18,110.0,790036,1996-07-18,NaN,1996-07-18
6,1996-07-18,91.8,790036,1996-07-18,NaN,1996-07-18
7,2000-08-24,187.0,860081,2000-08-24,NaN,NaT
8,2003-03-19,263.0,860081,2003-03-19,NaN,NaT
9,2007-03-29,260.0,860081,2007-03-29,NaN,NaT


In [ ]:
# nat in haem date  (matched on right col) mean that there was no match within tolerance

### next step:
#### do this on all 6 combos left and right.  

left bio, right haem
left bio, right urine

left haem, right bio
left haem, right urine

left urine, right bio
left urine, right haem

#### then come up with a scheme to find the smallest distance between all 3